In [ ]:
import os
import json
import yaml

import numpy as np

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import get_index, get_data_directory
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq

from pymedphys.level4.comparebygantry import (
    get_mappings, get_comparisons_byfield
)

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids_compared = set(cache.keys())

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields, field_id_grouped_hashes = get_mappings(index, file_hashes)

In [ ]:
patient_ids = set(patient_grouped_fields.keys())

In [ ]:
patient_ids_not_compared = patient_ids.difference(patient_ids_compared)

In [ ]:
for patient_id in patient_ids_not_compared:
    field_ids = patient_grouped_fields[patient_id]
    comparisons_byfield = get_comparisons_byfield(index, config, field_ids, field_id_grouped_hashes)
    
    cache[patient_id] = comparisons_byfield
    
    with open(cache_scratch_filepath, 'w') as cache_scratch_file:
        yaml.dump(cache, cache_scratch_file)
        
    os.replace(cache_scratch_filepath, cache_filepath)

In [ ]:
cache